In [12]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter # print to tensorboard
import torchvision

from IPython.core.magic import register_cell_magic
import requests

@register_cell_magic
def execute_remotely(line, cell):
    code = cell.strip()
    response = requests.post('http://192.168.0.105:2408/execute', data={'code': code})
    return response.text


In [13]:
# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Model constructor
class CNN(nn.Module):
    
    def __init__(self,n_channel,n_out):
        super(CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=n_channel,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1 = nn.Linear(in_features=16*7*7,out_features=n_out)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x =  F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.shape[0],-1)
        x = self.fc1(x)
        
        return x
        

In [15]:
path = "model_checkpoints/cnn-checkpoint.pth.tar"
def save_checkpoint(epoch,model,optimizer,loss,PATH=path):
    print('==> Saving Checkpoint')
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, PATH)
    
def load_checkpoint(model,optimizer,PATH=path):
    
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model , optimizer, epoch,loss
    
    

In [16]:
# Hyperparameters

n_channel = 1
n_out = 10
learing_rate = 0.01
n_epochs = 2
batch = 100
load_model = True


In [17]:
## Load Data
train_data = datasets.MNIST(root='datasets/',train= True,download=True, transform= transforms.ToTensor())
train_loader =DataLoader(train_data,batch,True)

test_data = datasets.MNIST(root='datasets/',train= False,download=True, transform= transforms.ToTensor())
test_loader =DataLoader(test_data,batch,True)

In [18]:
# Initialize Model 

model = CNN(n_channel,n_out).to(device= device)


In [19]:
## loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= learing_rate)
writer = SummaryWriter('runs/MNIST/trying_out_tensorboard')

In [20]:
last_trained_epoch = 0
if load_model:
    model, optimizer, last_trained_epoch, loss = load_checkpoint(model,optimizer)
    print('==> Load Model', )
    print(f'Last Trained:  epoch {last_trained_epoch}, loss {loss:.3f}')

==> Load Model
Last Trained:  epoch 14, loss 0.340


In [24]:
%%execute_remotely
batches= [128,1024]

learing_rate = [0.01,0.001]
classes = list('0123456789')


for batch in batches:
    for lr in learing_rate:
        train_loader =DataLoader(train_data,batch,True)
        model = CNN(n_channel,n_out).to(device= device)
        optimizer = optim.Adam(model.parameters(),lr= lr)
        writer = SummaryWriter(f'runs/MNIST/mnist batch {batch} lr {lr}')
        ## Train Network 
        step = 0
        for epoch in range(1,n_epochs):

            if (epoch % 2 == 0) and (epoch != 0):
                print(f'Loss at epoch {epoch} was {sum(losses)/len(losses):.3f}')
                save_checkpoint(last_trained_epoch+epoch,model,optimizer,sum(losses)/len(losses))

            losses = []
            accuracy = []


            for batch_idx, (data,target) in enumerate(train_loader):
                # Get data to cuda 
                data = data.to(device=device)
                target =target.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores,target)
                losses.append(loss.item())


                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent
                optimizer.step()
                


                # running training accuracy 
                _, predictions = scores.max(1)
                num_correct = (predictions == target).sum()
                running_acc = num_correct/data.shape[0]
                accuracy.append(running_acc)
                
                # print to tensorboard
                class_labels = [classes[label] for label in predictions]
                writer.add_scalar('Training Loss',loss,step)
                writer.add_scalar('Training Accuracy',running_acc,step)
                img_grid = torchvision.utils.make_grid(data)
                writer.add_image('mnist_images/',img_grid)
                writer.add_histogram('fc1',model.fc1.weight)
                writer.add_embedding(data.reshape(data.shape[0],-1),metadata=class_labels,label_img=data,global_step=step)
                step += 1
            writer.add_hparams({'batch':batch,'lr':lr},
                              {'accuracy':sum(accuracy)/len(accuracy),
                              'loss':sum(losses)/len(losses)})




'Executing remotly\n'

In [58]:
# check the accuracy of out trained model 
def check_accuracy(loader,model):
    for data,target in loader:
        num_correct = 0
        num_sample = 0
        model.eval()
        with torch.no_grad():
            data = data.to(device=device)
            target = target.to(device=device)


            scores = model(data)
            _, pred = scores.max(1)
            # print(list(zip(pred,target)))
            num_correct += sum(pred == target)
            num_sample  += pred.shape[0]
    print(f'Total {num_correct} correct  / out of {num_sample} - accuracy {num_correct/num_sample :.3f} ')
    model.train()
            

In [105]:
# on test dataset
check_accuracy(test_loader,model)

Total 99 correct  / out of 100 - accuracy 0.990 


In [106]:
# on train datasets
check_accuracy(train_loader,model)

Total 99 correct  / out of 100 - accuracy 0.990 
